## 데이터 수집_크롤링

In [1]:
#크롤링을 위한 라이브러리 import
import requests
import json
from bs4 import BeautifulSoup
import time

* 선수들의 결기 결과 정보를 수집 http://stats.nba.com/league/player/#!/

In [2]:
def nba_player_dataset(start_year, end_year):
    
    df = pd.DataFrame(columns=[     #Data Frame을 만들어 놓는다.
            'PLAYER_ID',
            'PLAYER_NAME',
            'TEAM_ID',
            'TEAM_ABBREVIATION',
            'AGE',
            'GP',
            'W',
            'L',
            'W_PCT',
            'MIN',
            'FGM',
            'FGA',
            'FG_PCT',
            'FG3M',
            'FG3A',
            'FG3_PCT',
            'FTM',
            'FTA',
            'FT_PCT',
            'OREB',
            'DREB',
            'REB',
            'AST',
            'TOV',
            'STL',
            'BLK',
            'BLKA',
            'PF',
            'PFD',
            'PTS',
            'PLUS_MINUS',
            'DD2',
            'TD3',
            'CFID',
            'CFPARAMS'
        ])

    headers = {"Host": "stats.nba.com",    
           "Connection": "keep-alive",     
           "Cache-Control": "max-age=0",
           "Upgrade-Insecure-Requests": "1",
           "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36",
           "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
           "Accept-Encoding": "gzip, deflate, sdch",
           "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4",
           "Cookie": "__gads=ID=55abf9f2f51740da:T=1468809285:S=ALNI_MY-dnsIhCT3AYpUecmlj0ZKqrMHgg; ug=578c40400b3d570a3c8de1775a0654eb; ugs=1; AMCV_248F210755B762187F000101%40AdobeOrg=793872103%7CMCAID%7C2BC6202185010E9D-6000014220004ADC%7CMCIDTS%7C17003%7CMCMID%7C65921844489051295175108060016669540424%7CMCAAMLH-1469663344%7C11%7CMCAAMB-1469663345%7Chmk_Lq6TPIBMW925SPhw3Q; _ga=GA1.2.1703639531.1468809282; crtg_trnr=; s_cc=true; s_sq=%5B%5BB%5D%5D; s_vi=[CS]v1|2BC6202185010E9D-6000014220004ADC[CE]; s_fid=5D7B5004A055748E-0F0FA2BE698255F7",
           }
   
    """
    JSONDecodeError: Expecting value: line 1 column 1 (char 0)
    headers를 지정하지 않으면 이러한 error가 나타날 것이다.
    
    #헤더 tuple을 지정한 이유는 응답 받을 때 요구 조건에 맞는 지 절차를 따르며 정확히 받아오기 위해서
    """
    
    for year in range(start_year, end_year+1):
        frames = []                          #빈 리스트에 해마다 불러온 데이터를 저장한다.
        year_plus_one = year + 1             #url 값을 계속해서 바꾸며 불러들이기 위해서
        year_plus_one = str(year_plus_one)   #str으로 바꾸지 않으면 연도를 int로 인식한다.
        year = str(year)                     #마찬가지. url 문자열과 같게 하기 위해서
        url = ""
        url = ("http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={start}-{end}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight="
        .format(start=year, end=year_plus_one[2:]))
        response = requests.get(url, headers=headers)     #헤더도 넣어주면 된다.
        json_info = response.json()                       #json으로 요청 받고
        nba_data = json_info["resultSets"][0]["rowSet"]   # 필요한 데이터를 찾아서 변수에 넣는다.
        
        for i, data in enumerate(nba_data):               #for문을 돌면서 df라는 데이터프레임에 하나씩 쌓는다.
                df.loc[i] = data
        
        frames.append(df)                                 #쌓은 df를 frames 리스트에 다시 쌓는다.
        
        result = pd.concat(frames)                        #쌓은 frames를 데이터프레임 형식으로 다시 불러와서
        result.to_csv("{start}_nba_player_data_set.csv".format(start=year))   #csv형태로 저장

In [38]:
nba_player_dataset(2005, 2011)   #오래 걸린다. 신중하게 선택해서 돌리자.

* 이번에는 연봉 데이터 수집
http://espn.go.com/nba/salaries

* 매년 전체 연봉 데이터를 mean값을 기준으로 비교해보면서

* 물가상승률의 영향력을 언제부터 받았는지 알아보고 이용여부를 판단해보자

In [3]:
def nba_salary_player(year):   #연봉 데이터만 수집하는 함수

    player_list = []
    
    for i in range(1, 12):
        url = "http://espn.go.com/nba/salaries/_/year/{year}/page/{i}/seasontype/4".format(year=year+1, i=i)
        response = requests.get(url)
        dom = BeautifulSoup(response.content, "html.parser")
        ranks = dom.select("tr.oddrow")                               #한 페이지에 홀수줄, 짝수줄 20개씩 나누어져 있어서 따로 가져와서
        for rank in ranks[0:20]:                                      #for문을 돌면서 name과 salary등 필요한 정보만 빈리스트에 추가
            name = str(rank.select_one("a")).split("</a>")[0].split(">")[1]
            salary = str(rank).split("</td></tr>")[0].split(">$")[1]
            player_list.append((name, salary))
        ranks = dom.select("tr.evenrow")
        for rank in ranks[0:20]:
            name = str(rank.select_one("a")).split("</a>")[0].split(">")[1]
            salary = str(rank).split("</td></tr>")[0].split(">$")[1]
            player_list.append((name, salary))    
    return player_list

* 연봉데이터가 x년도이면 선수데이터는 x-1년도로 해서 합친 DataFrame은 x-1년도로 통일해서 합치기

* 그렇게 활용한 연도의 데이터를 데이터프레임으로 만들겠다.

In [4]:
#2015년도의 결과와 2016년 연봉 데이터 합침 => test set
bin_list = nba_salary_player(2016)
salary_2016_df = pd.DataFrame(bin_list)
salary_2016_df.rename(columns={0: "PLAYER_NAME", 1: "Salary"}, inplace=True)
player_2015_df = pd.read_csv("./2015_nba_player_data_set.csv")
# tmp = player_2015_df["GP"] > 50   # => 이건 경기 수가 50경기 이상인 선수들만 뽑아서 해보았다,
# player_2015_df = player_2015_df[tmp]   # => 이유는 정규분포 형태 구해보려고 했으나 실패
player_salary_2015_df = player_2015_df.merge(salary_2016_df, on="PLAYER_NAME")
player_salary_2015_df["Salary"] = player_salary_2015_df["Salary"].apply(lambda x: np.int(x.replace(",", "")))

In [5]:
player_salary_2015_df["Salary"].mean(), len(player_salary_2015_df)
#원래는 test set 또한 물가가 비슷해야 하지만 크게 올랐다.
#그래서 2014년도 데이터로 시험해 본 결과 크게 달라지지 않았다. 물가상승률이 영향을 주는 것은 아니었다.

(7372670.757575758, 330)

* 무조건 합치는 것이 아니라 물가상승률을 고려하기 위하여

* 아래와 같은 과정을 연도 숫자를 바꿔가며 2006년까지 계속 반복했음

In [6]:
#2014년 데이터 프레임 만들어서
bin_list = nba_salary_player(2015)
salary_2015_df = pd.DataFrame(bin_list)
salary_2015_df.rename(columns={0: "PLAYER_NAME", 1: "Salary"}, inplace=True)
player_2014_df = pd.read_csv("./2014_nba_player_data_set.csv")
# tmp = player_2014_df["GP"] > 50
# player_2014_df = player_2014_df[tmp]
player_salary_2014_df = player_2014_df.merge(salary_2015_df, on="PLAYER_NAME")
player_salary_2014_df["Salary"] = player_salary_2014_df["Salary"].apply(lambda x: np.int(x.replace(",", "")))

In [7]:
player_salary_2014_df["Salary"].mean(), len(player_salary_2014_df)

(5472950.560117302, 341)

In [8]:
#2013년 데이터 프레임 만들어서
bin_list = nba_salary_player(2014)
salary_2014_df = pd.DataFrame(bin_list)
salary_2014_df.rename(columns={0: "PLAYER_NAME", 1: "Salary"}, inplace=True)
player_2013_df = pd.read_csv("./2013_nba_player_data_set.csv")
# tmp = player_2013_df["GP"] > 50
# player_2013_df = player_2013_df[tmp]
player_salary_2013_df = player_2013_df.merge(salary_2014_df, on="PLAYER_NAME")
player_salary_2013_df["Salary"] = player_salary_2013_df["Salary"].apply(lambda x: np.int(x.replace(",", "")))

In [9]:
player_salary_2013_df["Salary"].mean(), len(player_salary_2013_df)

(5075807.419354838, 341)

In [10]:
#train set 만들기 위한 과정
df_all = pd.concat([player_salary_2013_df, player_salary_2014_df], ignore_index=True)
#ignore_index=True한 이유? 인덱스 통일해야 합니다. 그래야 관리 용이

In [11]:
len(df_all)

682

In [12]:
bin_list = nba_salary_player(2013)   #2012년 추가
salary_2013_df = pd.DataFrame(bin_list)
salary_2013_df.rename(columns={0: "PLAYER_NAME", 1: "Salary"}, inplace=True)
player_2012_df = pd.read_csv("./2012_nba_player_data_set.csv")
# tmp = player_2012_df["GP"] > 50
# player_2012_df = player_2012_df[tmp]
player_salary_2012_df = player_2012_df.merge(salary_2013_df, on="PLAYER_NAME")
player_salary_2012_df["Salary"] = player_salary_2012_df["Salary"].apply(lambda x: np.int(x.replace(",", "")))

In [13]:
player_salary_2012_df["Salary"].mean(), len(player_salary_2012_df)

(5218590.015527951, 322)

In [14]:
df_all = pd.concat([player_salary_2012_df, df_all], ignore_index=True)
len(df_all)

1004

In [15]:
bin_list = nba_salary_player(2012)   #2011년 추가
salary_2012_df = pd.DataFrame(bin_list)
salary_2012_df.rename(columns={0: "PLAYER_NAME", 1: "Salary"}, inplace=True)
player_2011_df = pd.read_csv("./2011_nba_player_data_set.csv")
# tmp = player_2011_df["GP"] > 50
# player_2011_df = player_2011_df[tmp]
player_salary_2011_df = player_2011_df.merge(salary_2012_df, on="PLAYER_NAME")
player_salary_2011_df["Salary"] = player_salary_2011_df["Salary"].apply(lambda x: np.int(x.replace(",", "")))

In [16]:
player_salary_2011_df["Salary"].mean(), len(player_salary_2011_df)

(5396027.7192429025, 317)

In [17]:
df_all = pd.concat([player_salary_2011_df, df_all], ignore_index=True)
len(df_all)

1321

In [18]:
bin_list = nba_salary_player(2011)   #2010년 추가
salary_2011_df = pd.DataFrame(bin_list)
salary_2011_df.rename(columns={0: "PLAYER_NAME", 1: "Salary"}, inplace=True)
player_2010_df = pd.read_csv("./2010_nba_player_data_set.csv")
# tmp = player_2010_df["GP"] > 50
# player_2010_df = player_2010_df[tmp]
player_salary_2010_df = player_2010_df.merge(salary_2011_df, on="PLAYER_NAME")
player_salary_2010_df["Salary"] = player_salary_2010_df["Salary"].apply(lambda x: np.int(x.replace(",", "")))
player_salary_2010_df["Salary"].mean(), len(player_salary_2010_df)

(5328281.076246334, 341)

In [19]:
df_all = pd.concat([player_salary_2010_df, df_all], ignore_index=True)
len(df_all)

1662

In [20]:
bin_list = nba_salary_player(2010)   #2009년 추가
salary_2010_df = pd.DataFrame(bin_list)
salary_2010_df.rename(columns={0: "PLAYER_NAME", 1: "Salary"}, inplace=True)
player_2009_df = pd.read_csv("./2009_nba_player_data_set.csv")
# tmp = player_2009_df["GP"] > 50
# player_2009_df = player_2009_df[tmp]
player_salary_2009_df = player_2009_df.merge(salary_2010_df, on="PLAYER_NAME")
player_salary_2009_df["Salary"] = player_salary_2009_df["Salary"].apply(lambda x: np.int(x.replace(",", "")))
player_salary_2009_df["Salary"].mean(), len(player_salary_2009_df)

(5257192.629943503, 354)

In [21]:
df_all = pd.concat([player_salary_2009_df, df_all], ignore_index=True)
len(df_all)

2016

In [22]:
bin_list = nba_salary_player(2009)   #2008년 추가
salary_2009_df = pd.DataFrame(bin_list)
salary_2009_df.rename(columns={0: "PLAYER_NAME", 1: "Salary"}, inplace=True)
player_2008_df = pd.read_csv("./2008_nba_player_data_set.csv")
# tmp = player_2008_df["GP"] > 50
# player_2008_df = player_2008_df[tmp]
player_salary_2008_df = player_2008_df.merge(salary_2009_df, on="PLAYER_NAME")
player_salary_2008_df["Salary"] = player_salary_2008_df["Salary"].apply(lambda x: np.int(x.replace(",", "")))
player_salary_2008_df["Salary"].mean(), len(player_salary_2008_df)

(5469186.439093485, 353)

In [23]:
df_all = pd.concat([player_salary_2008_df, df_all], ignore_index=True)
len(df_all)

2369

In [24]:
bin_list = nba_salary_player(2008)   #2007년 추가
salary_2008_df = pd.DataFrame(bin_list)
salary_2008_df.rename(columns={0: "PLAYER_NAME", 1: "Salary"}, inplace=True)
player_2007_df = pd.read_csv("./2007_nba_player_data_set.csv")
# tmp = player_2007_df["GP"] > 50
# player_2007_df = player_2007_df[tmp]
player_salary_2007_df = player_2007_df.merge(salary_2008_df, on="PLAYER_NAME")
player_salary_2007_df["Salary"] = player_salary_2007_df["Salary"].apply(lambda x: np.int(x.replace(",", "")))
player_salary_2007_df["Salary"].mean(), len(player_salary_2007_df)

(5640090.470760234, 342)

In [25]:
df_all = pd.concat([player_salary_2007_df, df_all], ignore_index=True)
len(df_all)

2711

In [26]:
bin_list = nba_salary_player(2007)   #2006년은 확 떨어져서 배제
salary_2007_df = pd.DataFrame(bin_list)
salary_2007_df.rename(columns={0: "PLAYER_NAME", 1: "Salary"}, inplace=True)
player_2006_df = pd.read_csv("./2006_nba_player_data_set.csv")
# tmp = player_2006_df["GP"] > 50
# player_2006_df = player_2006_df[tmp]
player_salary_2006_df = player_2006_df.merge(salary_2007_df, on="PLAYER_NAME")
player_salary_2006_df["Salary"] = player_salary_2006_df["Salary"].apply(lambda x: np.int(x.replace(",", "")))
player_salary_2006_df["Salary"].mean(), len(player_salary_2006_df)

(5034361.794117647, 340)

In [27]:
bin_list = nba_salary_player(2006)   #2005년부터는 배제
salary_2006_df = pd.DataFrame(bin_list)
salary_2006_df.rename(columns={0: "PLAYER_NAME", 1: "Salary"}, inplace=True)
player_2005_df = pd.read_csv("./2005_nba_player_data_set.csv")
player_salary_2005_df = player_2005_df.merge(salary_2006_df, on="PLAYER_NAME")
player_salary_2005_df["Salary"] = player_salary_2005_df["Salary"].apply(lambda x: np.int(x.replace(",", "")))
player_salary_2005_df["Salary"].mean(), len(player_salary_2005_df)

(4756220.8537313435, 335)

* 총 정리하면 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007년 등 8년치의 데이터까지만 수집

* 선수 데이터는 총 2708명

* df_all.csv에 저장 후 활용

=================================================================================================

In [28]:
df_all.to_csv("df_all")

In [29]:
df_all = pd.read_csv("df_all")